In [ ]:
# Load library imports
import os
import ast
import folium
import logging
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pyproj import Transformer
from shapely.geometry import box
from datetime import datetime, timedelta

# Load project Imports
from src.utils.config_loader import load_project_config

In [ ]:
# Set up logging config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/eden_project_config.yaml")

Load gwl station list with grid references and convert grid references to easting, northing, longitude and latitude form for plotting and data alignment.

In [ ]:
# Use lookup file to create a dict of zipped grid letter and value lookup objects
grid_lookup_df = pd.read_csv("data/os_grid_squares.csv")
grid_letters = dict(zip(grid_lookup_df['grid_letters'], zip(grid_lookup_df['easting_base'], grid_lookup_df['northing_base'])))

# Apply to the station CSV
stations_df = pd.read_csv("data/station_list.csv")
stations_df[['easting', 'northing', 'lat', 'lon']] = stations_df['grid_ref'].apply(grid_ref_to_coords)

# Save as csv and check head and length
stations_df.to_csv("data/station_list_with_coords.csv", index=False)
print(f"Station location reference table:\n\n{stations_df.head()}")
print(f"\nTotal Stations: {len(stations_df)}")